In [ ]:
import os
import sys
os.environ['GLEAMS_HOME'] = os.path.join(os.environ['HOME'],
                                         'Projects/gleams')
# Make sure all code is in the PATH.
sys.path.append(
    os.path.normpath(os.path.join(os.environ['GLEAMS_HOME'], 'src')))

In [ ]:
import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.exceptions import EfficiencyWarning

In [ ]:
import logging
logger = logging.getLogger('gleams')
logger.setLevel(logging.DEBUG)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=EfficiencyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from gleams.dag import dag

from gleams import config
from gleams.cluster import cluster

In [ ]:
# Plot styling.
plt.style.use(['seaborn-white', 'seaborn-paper'])
plt.rc('font', family='serif')
sns.set_palette('Set1')
sns.set_context('paper', font_scale=1.3)    # Single-column figure.

In [ ]:
split = 'train'

In [ ]:
cluster.build_ann_index(
    os.path.join(os.environ['GLEAMS_HOME'], 'data', 'embed',
                 f'embed_{config.massivekb_task_id}_{split}.npy'))
cluster.compute_pairwise_distances(
    os.path.join(os.environ['GLEAMS_HOME'], 'data', 'embed',
                 f'embed_{config.massivekb_task_id}_{split}.npy'),
    os.path.join(os.environ['GLEAMS_HOME'], 'data', 'ann',
                 f'ann_{config.massivekb_task_id}_{split}.faiss'))

In [ ]:
def evaluate_clusters(clusters_filename: str, metadata_ident_filename: str,
                      metadata_all_filename: str,
                      min_peptide_size: int = None):
    clusters = pd.merge(pd.read_parquet(metadata_all_filename),
                        pd.read_parquet(metadata_ident_filename),
                        'left', ['dataset', 'filename', 'scan'])
    # Don't disambiguate between I/L.
    clusters['sequence'] = clusters['sequence'].str.replace('I', 'L')
    clusters['cluster'] = np.load(clusters_filename)
    # Only consider identified spectra as clustering ground truth.
    clusters_ident = clusters.dropna(subset=['sequence'])
    # Possibly only consider clusters of a minimal size.
    # (Is the clustering better for small/large clusters?)
    if min_peptide_size is not None:
        peptide_counts = clusters_ident['sequence'].value_counts()
        clusters_ident = clusters_ident[clusters_ident['sequence'].isin(
            peptide_counts[peptide_counts >= min_peptide_size].index)]
    clusters_ident_non_noise = clusters_ident[clusters_ident['cluster'] != -1]
    prop_clustered = len(clusters_ident_non_noise) / len(clusters_ident)
    prop_clustered_incorrect = (
            clusters_ident_non_noise.groupby('cluster')['sequence']
            .apply(lambda labels: len(labels) - labels.value_counts().iat[0])
            .sum()
            / len(clusters_ident))

    return prop_clustered, prop_clustered_incorrect

In [ ]:
cluster_filename = os.path.join(
    os.environ['GLEAMS_HOME'], 'data', 'ann',
    f'clusters_{config.massivekb_task_id}_{split}.npy')

In [ ]:
logger.info('Cluster hyperparameter search')
cluster_hyperparameter = [], [], [], []
min_peptide_size = 5
for eps in np.arange(0.01, 0.051, 0.005):
    for min_samples in [2, 3, 5, 10]:
        config.eps = eps
        config.min_samples = min_samples
        if os.path.isfile(cluster_filename):
            os.remove(cluster_filename)
        cluster.cluster(os.path.join(
            os.environ['GLEAMS_HOME'], 'data', 'ann',
            f'dist_{config.massivekb_task_id}_{split}.npz'))
        prop_clustered, prop_clustered_incorrect = evaluate_clusters(
            cluster_filename,
            os.path.join(os.environ['GLEAMS_HOME'], 'data', 'metadata',
                         f'metadata_{config.massivekb_task_id}_{split}.parquet'),
            os.path.join(os.environ['GLEAMS_HOME'], 'data', 'embed',
                         f'embed_{config.massivekb_task_id}_{split}.parquet'),
            min_peptide_size)
        logger.debug(f'Clustered = {prop_clustered:.2%}, '
                     f'incorrectly clustered = '
                     f'{prop_clustered_incorrect:.2%}')
        cluster_hyperparameter[0].append(config.eps)
        cluster_hyperparameter[1].append(config.min_samples)
        cluster_hyperparameter[2].append(prop_clustered)
        cluster_hyperparameter[3].append(prop_clustered_incorrect)
cluster_hyperparameter = pd.DataFrame(
    {'eps': cluster_hyperparameter[0],
     'min_samples': cluster_hyperparameter[1],
     'prop_clustered': cluster_hyperparameter[2],
     'prop_clustered_incorrect': cluster_hyperparameter[3]})

In [ ]:
# Remove final (suboptimal) clustering.
os.remove(cluster_filename)

In [ ]:
joblib.dump(cluster_hyperparameter, 'cluster_hyperparameter.joblib')

In [ ]:
cluster_hyperparameter = joblib.load('cluster_hyperparameter.joblib')

In [ ]:
def get_pareto_frontier(arr):
    # Sort by the first column.
    arr_sorted = arr[arr[:, 0].argsort()]
    # Iteratively add points to the Pareto frontier.
    pareto_idx = [0]
    for i in range(1, arr_sorted.shape[0]):
        if (arr_sorted[i, 0] > arr_sorted[pareto_idx[-1], 0] and
                arr_sorted[i, 1] > arr_sorted[pareto_idx[-1], 1]):
            pareto_idx.append(i)
    return arr_sorted[pareto_idx]

In [ ]:
acceptable_hyperparam = cluster_hyperparameter[
    cluster_hyperparameter['prop_clustered_incorrect'] < 0.01]
best_hyperparam = acceptable_hyperparam.iloc[
    acceptable_hyperparam['prop_clustered'].idxmax()]
print(f'Optimal clustering hyperparameters:\n'
      f'  - eps = {best_hyperparam["eps"]:.4f}\n'
      f'  - min_samples = {best_hyperparam["min_samples"]:.0f}\n'
      f'-> {best_hyperparam["prop_clustered"]:.2%} clustered, '
      f'{best_hyperparam["prop_clustered_incorrect"]:.2%} clustered incorrectly')

In [ ]:
width = 7
height = width / 1.618    # golden ratio
fig, ax = plt.subplots(figsize=(width, height))

clustering_pareto = get_pareto_frontier(np.column_stack(
    [cluster_hyperparameter['prop_clustered_incorrect'],
     cluster_hyperparameter['prop_clustered']]))
ax.plot(clustering_pareto[:, 0], clustering_pareto[:, 1], marker='o')
scatter = ax.scatter(cluster_hyperparameter['prop_clustered_incorrect'],
                     cluster_hyperparameter['prop_clustered'], marker='.')
ax.scatter(best_hyperparam['prop_clustered_incorrect'],
           best_hyperparam['prop_clustered'],
           s=200, c=scatter.get_facecolors(), marker='D')

# ax.set_xlim(-0.001, 0.015)
ax.set_ylim(-0.05, 1)

ax.set_xlabel('Incorrectly clustered spectra')
ax.set_ylabel('Clustered spectra')

ax.xaxis.set_major_formatter(mticker.PercentFormatter(1))
ax.yaxis.set_major_formatter(mticker.PercentFormatter(1))

sns.despine()

plt.savefig('cluster_hyperparameter.png', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
logging.shutdown()